# DS01 - Pandas 101

In this notebook we'll cover the following:

    - Dropping rows and columns 
    - Inplace
    - Copying dataframes
    - Basic math operations with dataframes
    - Group by (split/apply/combine)
    - Concat
    - Sorting the index
    - Setting the index
    - Resetting the index
    - Sorting values

We start by importing pandas:

In [ ]:
import pandas as pd

# This is an option to preview less rows in the notebook's cells' outputs
pd.options.display.max_rows = 6

# Read the data in file airbnb_rooms.csv into a pandas DataFrame and use column room_id as the DataFrame index.
df = pd.read_csv('data/airbnb_rooms.csv', index_col='room_id')

## Dropping rows and columns

In order to drop rows and columns from a DataFrame, we can use function [drop](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.drop.html).

In order to drop a row, we do the following:

In [ ]:
# This drops the row with index 17031
df.drop(labels=17031) # default axis=0

#If we want to drop multiple rows (or columns), we can use lists:
df.drop(labels=[6499, 17031])

# This drops column neighborhood
df.drop(columns='neighborhood') # or df.drop(labels='neighborhood', axis=1)


## Renaming columns

`df = df.rename(columns={'index': 'brewery_id'}) `


And another example with inplace=True:

## Copying DataFrames

When we're transforming DataFrames, it can be usefull to keep a copy of the original DataFrame.

We can do that with function [copy](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.copy.html):

In [ ]:
# copy function returns a copy of df
df_original = df.copy()

df_original.head()

## Basic math operations

### Between a constant and a DataFrame column

The operation is repeated for each row.

For example, if we want to compute the rooms' price per week (7 nights):

In [ ]:
# Creates a new column in the DataFrame (price_per_week), where each row is equal to the price * 7
df['price_per_week'] = df.price * 7
df.head()

### Between a two DataFrame columns

The operation is performed element-wise, i.e, for each row, we apply the operation between the two columns' values.

For instance, if we want to compute the people per bedroom ratio in each room:

In [ ]:
# Creates a new column in the DataFrame (people_per_bedroom), 
# where each row is equal to the value of the accommodates column divided by the bedrooms column
df['people_per_bedroom'] = df.accommodates / df.bedrooms

df.head()

In [ ]:
def myfunc(row):
    #print (row)
    return row["host_id"] + row["accommodates"]* row["price"]

df["new_col"] = df.apply(myfunc, axis='columns')
df.head()

## Group by

This is a very extensive topic, and we'll just touch it's surface here, so that you know that exists and can explore it further later by your own.

In case you've worked with SQL before, you'll find this very familiar :)

So, in Pandas there is process a of three chained steps called [split-apply-combine](https://pandas.pydata.org/pandas-docs/stable/groupby.html):
* __split__: splitting the DataFrame into groups (this is the groupby)
* __apply__: apply a function to each group (aggregation, transformation and filtration)
* __combine__: create a DataFrame with the results

Let's see an example!

We want to find how many rooms each landlord has.
For that, consider this smaller DataFrame:

In [ ]:
df_smaller = pd.read_csv('data/airbnb_groupby.csv', index_col='room_id')

df_smaller

So the first step is to group our data by 'host_id'. This returns a DataFrameGroupBy object that by itself doesn't tell us much.

However, we can use the group property of the DataFrameGroupBy object to inspect the groups.

In [ ]:
df_grouped_by_host_id = df_smaller.groupby('host_id')

df_grouped_by_host_id

In [ ]:
df_grouped_by_host_id.groups

Then, the next step is to apply a function to each group, that aggregates our grouped data. In this case, we want to find the size of each group, i.e, the number of room in each group:

In [ ]:
df_grouped_by_host_id.size()

Another example. Let's find out what's the average price per room and the maximum number of bedrooms per room in our original dataset.

In [ ]:
# Read the dataset again, as we lost the neighborhood column along the way...
df = pd.read_csv('data/airbnb_rooms.csv', index_col='room_id')

# Group data by neighborhood
df_grouped = df.groupby('neighborhood')

# Aggregate the price using the average function and aggregate the bedrooms with the max function
df_grouped.agg({'price': ['mean'], 'bedrooms': ['max']})

## Concat

The [concat](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.concat.html) function can be used to concatenate DataFrames, either along the rows or the columns.

Let's see some examples.

Imagine we have two DataFrames, one with rooms in Areeiro, and the other with rooms in Benfica.
And now we want to concatenate the two DataFrames in order to have a unique DataFrame with all the rooms.

In [ ]:
# Get all the rooms in Areeiro
df_areeiro = df[df.neighborhood == 'Areeiro']
print('We have {} rooms in Areeiro'.format(len(df_areeiro)))

# Get all the rooms in Benfica
df_benfica = df[df.neighborhood == 'Benfica']
print('We have {} rooms in Benfica'.format(len(df_benfica)))

# Create a unique DataFrame by concatenating df_areeeiro and df_benfica
df_areeiro_benfica = pd.concat([df_areeiro, df_benfica])

df_areeiro_benfica

Let's get some more data from file __airbnb_locations.csv__.

This dataset has the coordinates for each room.

In [ ]:
df_locations = pd.read_csv('data/airbnb_locations.csv', index_col='room_id')

df_locations

Our next example is to concatenate df_areeiro_benfica and df_locations on the columns. I.e, we want to add columns __latitude__ and __longitude__ to df_areeiro_benfica.

In order to tell function concat that we want to perform the concatenation along the columns, we use the __axis=1__ parameter.

In [ ]:
# Concatenate DataFrames df_areeiro_benfica and df_locations along the columns
pd.concat([df_areeiro_benfica, df_locations], axis='columns')

But... What are all those NaN values??

Well, there are certain indexes that were only found in df_locations (and not in df_areeiro_benfica).
So, in this cases, the concat function fills the missing values with NaN.

And what if we only want to keep the rooms that exist in both DataFrames? We use the __join='inner'__ parameter.

In [ ]:
# Concatenate DataFrames df_areeiro_benfica and df_locations along the columns,
# only keeping rows that exists in both DataFrames
pd.concat([df_areeiro_benfica, df_locations], axis='columns', join='inner')

## Sorting the index

With the [sort_index](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.sort_index.html) function, we can sort the DataFrame along the index.

For instance, our DataFrame df was already sorted along the index, but we can resort it from bigger to smaller rooms ids, using the __ascending=False__ parameter.

In [ ]:
# Original df
df.head()

In [ ]:
# df with the index sorted from bigger to smaller room_id
df.sort_index(ascending=False)

## Resetting the index

We can reset the index of a DataFrame with function [reset_index](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.reset_index.html).
This will convert the index into a range from 0 to the length of the DataFrame minus 1.

Regarding the old index, we can either keep it by adding it as a column in the DataFrame (__drop=False__, this is the default behaviour) or reset it completely (__drop=True__).

In [ ]:
# Resetting the index and keeping it as a new column room_id
df.reset_index()

In [ ]:
# Resetting the index and dropping it -> no new column is added
df.reset_index(drop=True)

## Setting the index

With function [set_index](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.set_index.html), we can set a new index for our DataFrame.

The old index is dropped.

In this function, the __drop=True__ parameter deletes the column to be used as the new index, which is the default behaviour, and __drop=False__ keeps the column unchanged.

In [ ]:
# Setting column neighborhood as the new index
# The neighborhood column is dropped from the DataFrame, this is the default behaviour
df.set_index('neighborhood', drop=True)

In [ ]:
# Setting column neighborhood as the new index
# The neighborhood column is NOT dropped from the DataFrame
df.set_index('neighborhood', drop=False)

## Sorting values

Function [sort_values](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.sort_values.html) can be used to sort the DataFrame along a certain column.

For instance, let's sort df from cheapest to more expensive rooms:

In [ ]:
df.sort_values(by='price')